<a href="https://colab.research.google.com/github/fcoelopes/ImersaoIAAlura/blob/main/busca_em_documentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -q -U google-generativeai

Pacotes que serão requeridos no desenvolvimento do projeto

In [2]:
import google.generativeai as genai
import pandas as pd
import numpy as np

Configuração da chave de API

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Lista de modelos

In [4]:
for models in genai.list_models():
  if 'generateContent' in models.supported_generation_methods:
    print (models.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


Lista de embeds disponíveis

In [5]:
for embeds in genai.list_models():
  if 'embedContent' in embeds.supported_generation_methods:
    print (embeds.name)

models/embedding-001
models/text-embedding-004


In [6]:
embed_model = "models/embedding-001"

Configurando o modelo

In [7]:
generation_config = {
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 0,

}

In [8]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    }
]

In [9]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              generation_config= generation_config,
                              safety_settings= safety_settings)

Cria um embedding a partir de um texto

In [10]:
title = "The next generation of AI for developers and Google Workspace"
sample_text = ("Title: The next generation of AI for developers and Google Workspace"
    "\n"
    "Full article:\n"
    "\n"
    "Gemini API & Google AI Studio: An approachable way to explore and prototype with generative AI applications")

embedding = genai.embed_content(model=embed_model,
                                content=sample_text,
                                task_type="retrieval_document",
                                title=title)

print(str(embedding["embedding"])[:50], "... TRIMMED]")

[0.03411343, -0.05517662, -0.020209055, -0.0041249 ... TRIMMED]


Cria um banco de dados de embeddings

In [11]:
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [12]:
df = pd.DataFrame(documents)
df.columns = ['Titulo', 'Texto']
df

,Titulo,Texto
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


Função para criar embeddings dos documentos

In [13]:
def gera_embedding(title, text):
  return genai.embed_content(model=embed_model,
                                content=text,
                                task_type="RETRIEVAL_DOCUMENT",
                                title=title)["embedding"]

In [14]:
df["Embedding"] = df.apply(lambda row: gera_embedding(row["Titulo"], row["Texto"]), axis=1)
df

,Titulo,Texto,Embedding
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[-0.033361107, -0.021217084, -0.049581926, -0...."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.009660736, -0.030662702, -0.017281422, -0.0..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[-0.04270796, -0.007160868, -0.03242516, -0.02..."


Função para criar embedding do prompt do usuário

In [15]:
def gera_embedding_consulta(consulta, base, model):
  embedding_consulta = genai.embed_content(model=embed_model,
                                           content=consulta,
                                           task_type="retrieval_query")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embedding"]), embedding_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Texto"]


In [16]:
consulta = input("Aguardando prompt:")
trecho = gera_embedding_consulta(consulta, df, model)
print(trecho)

Aguardando prompt:Gear
Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions.


In [17]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"
response = model.generate_content(prompt)
print(response.text)

**Guia de marchas do seu Googlecar:**

* **Estacionado:** Quando você estaciona, é como se o carro estivesse dormindo. As rodas ficam travadas e ele não se mexe.
* **Ré:** Para dar aquela rézinha, é só colocar o câmbio nessa posição.
* **Neutro:** Quando você para no sinal ou no trânsito, é como se o carro estivesse de bobeira. Ele não está engatado e só vai andar se você pisar no acelerador.
* **Dirigir:** Essa é a marcha para você sair por aí.
* **Baixa:** Se você estiver dirigindo na neve ou em lugares escorregadios, essa marcha é a sua amiga.


Inicializando o chat

In [18]:
chat = model.start_chat(history=[])

In [19]:
consulta = input("Aguardando prompt:")
trecho = gera_embedding_consulta(consulta, df, model)

while consulta != "Interromper":
  response = chat.send_message(trecho)
  print(response.text)
  print(f"{model.count_tokens(prompt)}/{model.count_tokens(chat.history)}")
  consulta = input("Aguardando prompt:")
  trecho = gera_embedding_consulta(consulta, df, model)

Aguardando prompt:Gears
**Park:**
- Used when the car is parked.
- Wheels are locked, preventing movement.

**Reverse:**
- Used for backing up.

**Neutral:**
- Used when the car is stopped at a light or in traffic.
- Car is not in gear and will not move unless the gas pedal is pressed.

**Drive:**
- Used for driving forward.

**Low:**
- Used for driving in snow or other slippery conditions.
total_tokens: 137
/total_tokens: 211

Aguardando prompt:Interromper
